In [1]:
import numpy as np
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
from elfi.examples.gauss import get_model

%matplotlib inline
%precision 2

import logging
logging.basicConfig(level=logging.INFO)

# Set an arbitrary global seed to keep the randomly generated quantities the same
seed = 1
np.random.seed(seed)

import elfi
N = 1000
fitting_steps = 25

### 1-D Gaussian model

In [2]:
true_mu_1d = [2]
cov_matrix_1d = [1]

In [3]:
m_1d = get_model(true_params=true_mu_1d, nd_mean=True, cov_matrix=cov_matrix_1d, seed_obs=seed)

In [4]:
m_1d.parameter_names

['mu_0']

In [5]:
rej_1d = elfi.Rejection(m_1d['d'], batch_size=N, seed=seed)

In [6]:
bounds_1d = {'mu_0': (-2.5, 2.5)}

In [7]:
gp_1d = elfi.GPyRegression(m_1d.parameter_names, bounds_1d)

In [8]:
prior_1d = elfi.methods.utils.ModelPrior(m_1d)

In [9]:
acq_1d = elfi.methods.bo.acquisition.RandMaxVar(model=gp_1d, prior=prior_1d)

In [10]:
initial_evidence = 10
update_interval = 1
n_rejection_samples = 2 * initial_evidence

In [11]:
rej_result_1d = rej_1d.sample(n_samples=n_rejection_samples, quantile = 1)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [12]:
bolfi_1d = elfi.BOLFI(m_1d, 'd', batch_size=1, initial_evidence=rej_result_1d.outputs, 
                   update_interval=update_interval, bounds=bounds_1d, seed=scipy.random.seed(), 
                   async=True, target_model=gp_1d, acquisition_method=acq_1d, acq_noise_var=[0.1], 
                   batches_per_acquisition=1)

In [13]:
bolfi_1d.fit(n_evidence=fitting_steps)

INFO:elfi.methods.parameter_inference:BOLFI: Fitting the surrogate model...
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


 /m/home/home2/27/musaevm1/unix/Documents/RA/elfi/elfi/methods/mcmc.py:207: RuntimeWarning:overflow encountered in exp
INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.039. After warmup 15 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.055. After warmup 15 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.500
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.658
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.962


Progress: |██████████████████████████████████████████████████| 100.0% Complete


INFO:elfi.methods.posteriors:Using optimized minimum value (0.0419) of the GP discrepancy mean function as a threshold


### 2-D Gaussian model

In [14]:
true_mu_2d = [.2, .3]
cov_matrix_2d = [[.3, .4], [.1, .2]]

In [15]:
m_2d = get_model(true_params=true_mu_2d, nd_mean=True, cov_matrix=cov_matrix_2d, seed_obs=seed)

 /l/musaevm1/anaconda3/elfi_test/lib/python3.6/site-packages/scipy/stats/_multivariate.py:652: RuntimeWarning:covariance is not positive-semidefinite.


In [16]:
m_2d.parameter_names

['mu_0', 'mu_1']

In [17]:
bounds_2d = {'mu_0': (-.25, .25), 'mu_1': (-.5, 5)}

In [18]:
gp_2d = elfi.GPyRegression(m_2d.parameter_names, bounds_2d)

In [19]:
prior_2d = elfi.methods.utils.ModelPrior(m_2d)

In [20]:
acq_2d = elfi.methods.bo.acquisition.RandMaxVar(model=gp_2d, prior=prior_2d)

In [21]:
rej_2d = elfi.Rejection(m_2d['d'], batch_size = 1)

In [22]:
rej_result_2d = rej_2d.sample(n_samples=n_rejection_samples, quantile = 1)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


 /l/musaevm1/anaconda3/elfi_test/lib/python3.6/site-packages/scipy/stats/_multivariate.py:652: RuntimeWarning:covariance is not positive-semidefinite.


In [23]:
bolfi_2d = elfi.BOLFI(m_2d, 'd', batch_size=1, initial_evidence=rej_result_2d.outputs, 
                   update_interval=update_interval, bounds=bounds_2d, seed=scipy.random.seed(), 
                   async=True, target_model=gp_2d, acquisition_method=acq_2d, acq_noise_var=[0.1, 0.01], 
                   batches_per_acquisition=1)

In [24]:
bolfi_2d.fit(n_evidence=fitting_steps)

INFO:elfi.methods.parameter_inference:BOLFI: Fitting the surrogate model...
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.391. After warmup 4 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
 /l/musaevm1/anaconda3/elfi_test/lib/python3.6/site-packages/scipy/stats/_multivariate.py:652: RuntimeWarning:covariance is not positive-semidefinite.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.187
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.410. After warmup 21 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.
 /m/home/home2/27/musaevm1/unix/Documents/RA/elfi/elfi/methods/mcmc.py:207: RuntimeWarning:overflow encountered in exp


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.272. After warmup 1 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.164


Progress: |██████████████████████████████████████████████████| 100.0% Complete


INFO:elfi.methods.posteriors:Using optimized minimum value (0.5896) of the GP discrepancy mean function as a threshold


### 3-D Gaussian model

In [25]:
true_mu_3d = [.2, .5, .6]
cov_matrix_3d = [[.3, .4, .7], [.1, .3, .2], [.8, .9, .1]]

In [26]:
m_3d = get_model(true_params=true_mu_3d, nd_mean=True, cov_matrix=cov_matrix_3d, seed_obs=seed)

 /l/musaevm1/anaconda3/elfi_test/lib/python3.6/site-packages/scipy/stats/_multivariate.py:652: RuntimeWarning:covariance is not positive-semidefinite.


In [27]:
m_3d.parameter_names

['mu_0', 'mu_1', 'mu_2']

In [28]:
rej_3d = elfi.Rejection(m_3d['d'], batch_size=N, seed=seed)

In [29]:
bounds_3d = {'mu_0': (-.25, .25), 'mu_1': (-.55, .55), 'mu_2': (-.65, .65)}

In [30]:
gp_3d = elfi.GPyRegression(m_3d.parameter_names, bounds_3d)

In [31]:
prior_3d = elfi.methods.utils.ModelPrior(m_3d)

In [32]:
acq_3d = elfi.methods.bo.acquisition.RandMaxVar(model=gp_3d, prior=prior_3d)

In [33]:
rej_result_3d = rej_3d.sample(n_samples=n_rejection_samples, quantile = 1)

 /l/musaevm1/anaconda3/elfi_test/lib/python3.6/site-packages/scipy/stats/_multivariate.py:652: RuntimeWarning:covariance is not positive-semidefinite.


Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [34]:
bolfi_3d = elfi.BOLFI(m_3d, 'd', batch_size=1, initial_evidence=rej_result_3d.outputs, 
                   update_interval=update_interval, bounds=bounds_3d, seed=scipy.random.seed(), 
                   async=True, target_model=gp_3d, acquisition_method=acq_3d, acq_noise_var=[0.1, 0.01, 0.1], 
                   batches_per_acquisition=1)

In [35]:
post = bolfi_3d.fit(n_evidence=fitting_steps)

INFO:elfi.methods.parameter_inference:BOLFI: Fitting the surrogate model...
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.123. After warmup 24 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
 /l/musaevm1/anaconda3/elfi_test/lib/python3.6/site-packages/scipy/stats/_multivariate.py:652: RuntimeWarning:covariance is not positive-semidefinite.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.227. After warmup 24 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.022. After warmup 9 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.149. After warmup 17 proposals were outside of the region allowed by priors and rejected, decreasing acceptance ratio.
INFO:elfi.methods.mcmc:NUTS: Performing 50 iterations with 25 adaptation steps.


INFO:elfi.methods.mcmc:NUTS: Adaptation/warmup finished. Sampling...
INFO:elfi.methods.mcmc:NUTS: Acceptance ratio: 0.171


Progress: |██████████████████████████████████████████████████| 100.0% Complete


INFO:elfi.methods.posteriors:Using optimized minimum value (0.5568) of the GP discrepancy mean function as a threshold
